In [1]:
# imports 
import numpy as np
import pandas as pd
from math import sqrt 
from pprint import pprint
from numpy import array

In [2]:
# load data
import constants
df = pd.read_csv("kddcup.data_10_percent.gz", names = constants.names)

In [3]:
# Categorical features to numeric labels

from sklearn import preprocessing
le_dicts = {}

for categorical_name in constants.categorical_names:
    le = preprocessing.LabelEncoder()
    le.fit(df[categorical_name])
    le_dicts[categorical_name] = dict(zip(le.transform(le.classes_), le.classes_))
    df[categorical_name + '_num'] = le.fit_transform(df[categorical_name])

df['cible'] = df.label.apply(lambda label: 1 if label == 'normal.' else 0)
df['label_four'] = df.label.apply(lambda label: constants.label_to_four_attack_class[label])
df['label_four_num'] = df.label_four.apply(lambda label: constants.five_classes_to_num[label])
df.drop(constants.categorical_names + ['label','label_four_num','label_four'], axis=1, inplace=True)
df.shape

(494021, 42)

In [4]:
# head of data (5)
df.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,protocol_type_num,service_num,flag_num,cible
0,0,181,5450,0,0,0,0,0,1,0,...,0.11,0.0,0.0,0.0,0.0,0.0,1,22,9,1
1,0,239,486,0,0,0,0,0,1,0,...,0.05,0.0,0.0,0.0,0.0,0.0,1,22,9,1
2,0,235,1337,0,0,0,0,0,1,0,...,0.03,0.0,0.0,0.0,0.0,0.0,1,22,9,1
3,0,219,1337,0,0,0,0,0,1,0,...,0.03,0.0,0.0,0.0,0.0,0.0,1,22,9,1
4,0,217,2032,0,0,0,0,0,1,0,...,0.02,0.0,0.0,0.0,0.0,0.0,1,22,9,1


In [5]:
# data to values
df = df.values

In [6]:
# Deep Extratrees
import argparse
import numpy as np
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/kdd.json")
gc = GCForest(config)

In [8]:
# train test
from sklearn.cross_validation import train_test_split
y = df[:,41]
X = df[:,0:41]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [9]:
# count class
len(np.unique(y_train))

2

In [10]:
 # X_enc is the concatenated predict_proba result of ExtraTrees
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    

[ 2018-04-20 21:20:17,108][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 41)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 41)],y_test.shape=(163027,)
[ 2018-04-20 21:20:17,172][cascade_classifier.fit_transform] group_dims=[41]
[ 2018-04-20 21:20:17,173][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-20 21:20:17,174][cascade_classifier.fit_transform] group_ends=[41]
[ 2018-04-20 21:20:17,175][cascade_classifier.fit_transform] X_train.shape=(330994, 41),X_test.shape=(163027, 41)
[ 2018-04-20 21:20:17,246][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 41), X_cur_test.shape=(163027, 41)
[ 2018-04-20 21:20:19,627][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.97%
[ 2018-04-20 21:20:22,034][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.96%
[ 2018-04-20 21:20:24,851][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

In [11]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of Extratrees = {:.6f} %".format(acc * 100))

[ 2018-04-20 21:20:47,012][cascade_classifier.transform] X_groups_test.shape=[(163027, 41)]
[ 2018-04-20 21:20:47,041][cascade_classifier.transform] group_dims=[41]
[ 2018-04-20 21:20:47,042][cascade_classifier.transform] X_test.shape=(163027, 41)
[ 2018-04-20 21:20:47,066][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(163027, 41)


Test Accuracy of Extratrees = 99.976078 %


In [12]:
X_train_enc.shape

(330994, 2)

In [13]:
X_test_enc.shape

(163027, 2)

In [14]:
X_train.shape

(330994, 41)

In [15]:
 # predict_proba result of ExtraTrees + data
    
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-20 21:20:51,150][cascade_classifier.transform] X_groups_test.shape=[(163027, 41)]
[ 2018-04-20 21:20:51,174][cascade_classifier.transform] group_dims=[41]
[ 2018-04-20 21:20:51,175][cascade_classifier.transform] X_test.shape=(163027, 41)
[ 2018-04-20 21:20:51,197][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(163027, 41)


In [16]:
X_train_new = X_train_enc
X_test_new  = X_test_enc

In [17]:
# Cascade ExtraTrees
config = load_json("./examples/kddgc.json")  
gc = GCForest(config)

X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-20 21:20:55,721][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 43)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 43)],y_test.shape=(163027,)
[ 2018-04-20 21:20:55,796][cascade_classifier.fit_transform] group_dims=[43]
[ 2018-04-20 21:20:55,797][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-20 21:20:55,798][cascade_classifier.fit_transform] group_ends=[43]
[ 2018-04-20 21:20:55,799][cascade_classifier.fit_transform] X_train.shape=(330994, 43),X_test.shape=(163027, 43)
[ 2018-04-20 21:20:55,879][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 43), X_cur_test.shape=(163027, 43)
[ 2018-04-20 21:20:58,165][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.96%
[ 2018-04-20 21:21:00,393][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.96%
[ 2018-04-20 21:21:03,002][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

[ 2018-04-20 21:22:47,042][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=99.98%
[ 2018-04-20 21:22:49,402][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=99.99%
[ 2018-04-20 21:22:51,707][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=99.98%
[ 2018-04-20 21:22:54,123][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=99.97%
[ 2018-04-20 21:22:56,682][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=99.96%
[ 2018-04-20 21:22:59,126][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=99.97%
[ 2018-04-20 21:23:01,992][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=99.97%
[ 2018-04-20 21:23:04,877][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=99.98%


In [18]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of Extratrees = {:.6f} %".format(acc * 100),i,'fois')

In [19]:
# deep
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    
    config = load_json("./examples/kddgc.json")  
    gc = GCForest(config)
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()    

[ 2018-04-20 21:23:35,042][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 43)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 43)],y_test.shape=(163027,)
[ 2018-04-20 21:23:35,109][cascade_classifier.fit_transform] group_dims=[43]
[ 2018-04-20 21:23:35,110][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-20 21:23:35,112][cascade_classifier.fit_transform] group_ends=[43]
[ 2018-04-20 21:23:35,113][cascade_classifier.fit_transform] X_train.shape=(330994, 43),X_test.shape=(163027, 43)
[ 2018-04-20 21:23:35,187][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 43), X_cur_test.shape=(163027, 43)
[ 2018-04-20 21:23:37,531][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.95%
[ 2018-04-20 21:23:40,046][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.96%
[ 2018-04-20 21:23:42,540][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

[ 2018-04-20 21:25:29,189][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=99.98%
[ 2018-04-20 21:25:32,223][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=99.99%
[ 2018-04-20 21:25:34,794][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=99.98%
[ 2018-04-20 21:25:37,433][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=99.97%
[ 2018-04-20 21:25:40,213][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=99.95%
[ 2018-04-20 21:25:43,014][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=99.97%
[ 2018-04-20 21:25:45,726][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=99.97%
[ 2018-04-20 21:25:48,218][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=99.97%


[ 2018-04-20 21:27:32,325][kfold_wrapper.log_eval_metrics] Accuracy(layer_8 - estimator_0 - 10_folds.train_7.predict)=99.97%
[ 2018-04-20 21:27:35,148][kfold_wrapper.log_eval_metrics] Accuracy(layer_8 - estimator_0 - 10_folds.train_8.predict)=99.99%
[ 2018-04-20 21:27:37,925][kfold_wrapper.log_eval_metrics] Accuracy(layer_8 - estimator_0 - 10_folds.train_9.predict)=99.98%
[ 2018-04-20 21:27:38,243][kfold_wrapper.log_eval_metrics] Accuracy(layer_8 - estimator_0 - 10_folds.train_cv.predict)=99.97%
[ 2018-04-20 21:27:38,246][kfold_wrapper.log_eval_metrics] Accuracy(layer_8 - estimator_0 - 10_folds.test.predict)=99.97%
[ 2018-04-20 21:27:38,257][cascade_classifier.calc_accuracy] Accuracy(layer_8 - train.classifier_average)=99.97%
[ 2018-04-20 21:27:38,261][cascade_classifier.calc_accuracy] Accuracy(layer_8 - test.classifier_average)=99.97%
[ 2018-04-20 21:27:38,354][cascade_classifier.fit_transform] [layer=9] look_indexs=[0], X_cur_train.shape=(330994, 45), X_cur_test.shape=(163027, 45)
[ 

('Test Accuracy of Extratrees = 99.976078 %', 0, 'fois')


[ 2018-04-20 21:28:35,174][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 43)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 43)],y_test.shape=(163027,)
[ 2018-04-20 21:28:35,241][cascade_classifier.fit_transform] group_dims=[43]
[ 2018-04-20 21:28:35,242][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-20 21:28:35,243][cascade_classifier.fit_transform] group_ends=[43]
[ 2018-04-20 21:28:35,245][cascade_classifier.fit_transform] X_train.shape=(330994, 43),X_test.shape=(163027, 43)
[ 2018-04-20 21:28:35,320][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 43), X_cur_test.shape=(163027, 43)
[ 2018-04-20 21:28:37,710][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.96%
[ 2018-04-20 21:28:40,320][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.96%
[ 2018-04-20 21:28:43,392][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

...

[ 2018-04-20 22:20:59,130][cascade_classifier.transform] group_dims=[43]
[ 2018-04-20 22:20:59,133][cascade_classifier.transform] X_test.shape=(163027, 43)
[ 2018-04-20 22:20:59,157][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(163027, 43)


('Test Accuracy of Extratrees = 99.974237 %', 18, 'fois')


[ 2018-04-20 22:21:03,775][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 43)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 43)],y_test.shape=(163027,)
[ 2018-04-20 22:21:03,844][cascade_classifier.fit_transform] group_dims=[43]
[ 2018-04-20 22:21:03,846][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-20 22:21:03,847][cascade_classifier.fit_transform] group_ends=[43]
[ 2018-04-20 22:21:03,848][cascade_classifier.fit_transform] X_train.shape=(330994, 43),X_test.shape=(163027, 43)
[ 2018-04-20 22:21:03,925][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 43), X_cur_test.shape=(163027, 43)
[ 2018-04-20 22:21:06,330][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.96%
[ 2018-04-20 22:21:09,029][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.96%
[ 2018-04-20 22:21:11,752][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

[ 2018-04-20 22:22:58,169][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=99.97%
[ 2018-04-20 22:23:00,953][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=99.97%
[ 2018-04-20 22:23:03,577][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=99.98%
[ 2018-04-20 22:23:06,572][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=99.98%
[ 2018-04-20 22:23:09,540][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=99.97%
[ 2018-04-20 22:23:12,177][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=99.96%
[ 2018-04-20 22:23:14,985][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=99.96%
[ 2018-04-20 22:23:17,804][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=99.96%


('Test Accuracy of Extratrees = 99.970557 %', 19, 'fois')


# ('Test Accuracy of Extratrees = 99.976078 %', 0, 'fois')